In [ ]:
from scipy import stats
import dautil as dl
from dautil.stats import jackknife
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.html.widgets.interaction import interact
from IPython.display import HTML

In [ ]:
def set_labels(ax, var):
    ax.set_xlabel('Day of year')
    ax.set_ylabel(dl.data.Weather.get_header(var))

In [ ]:
def plot_bayes(ax, metric, var, df):
    vals = np.array([[v.statistic, v.minmax[0], v.minmax[1]] for v in
                     df[metric].values])

    ax.set_title('Bayes {}'.format(metric))
    ax.errorbar(np.arange(len(vals)), vals.T[0], yerr=(vals.T[1], vals.T[2]))
    set_labels(ax, var)

In [ ]:
def plot_jackknife(ax, metric, func, var, df):
    vals = df.apply(lambda x: jackknife(x, func, alpha=0.95))
    vals = np.array([[v[0], v[1], v[2]] for v in vals.values])

    ax.set_title('Jackknife {}'.format(metric))
    ax.errorbar(np.arange(len(vals)), vals.T[0], yerr=(vals.T[1], vals.T[2]))
    set_labels(ax, var)

In [ ]:
def confidence_interval(var='TEMP'):
    df = dl.data.Weather.load().dropna()
    df = dl.ts.groupby_yday(df)
    
    def f(x):
        return stats.bayes_mvs(x, alpha=0.95)

    bayes_df = pd.DataFrame([[v[0], v[1], v[2]] for v in
                             df[var].apply(f).values], columns=['Mean', 'Var',
                                                                'Std'])

    fig, axes = plt.subplots(2, 2)
    fig.suptitle('Confidence Intervals')

    plot_bayes(axes[0][0], 'Mean', var, bayes_df)
    plot_bayes(axes[0][1], 'Var', var, bayes_df)
    plot_jackknife(axes[1][0], 'Mean', np.mean, var, df[var])
    plot_jackknife(axes[1][1], 'Mean', np.var, var, df[var])

    plt.tight_layout()

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
context = dl.nb.Context('bayes_confidence')
dl.nb.RcWidget(context)

In [ ]:
interact(confidence_interval, var=dl.data.Weather.get_headers())
HTML(dl.report.HTMLBuilder().watermark())